In [1]:
import sqlite3
import pandas as pd
import numpy as np 

<h1> Rt Reviews Dataset

In [125]:
df_rtreview_info = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t',encoding='windows-1252')
df_rtreview_info

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


<h3> Reviews Info

In [39]:
df_rtreview_info.describe()

## Top critic could be converted to Y or N, Rating could be renamed to "Rating_Score5" and be converted to an integer

,id,top_critic
count,54432.000000,54432.000000
mean,1045.706882,0.240594
std,586.657046,0.427448
min,3.000000,0.000000
25%,542.000000,0.000000
50%,1083.000000,0.000000
75%,1541.000000,0.000000
max,2000.000000,1.000000


<h5> Find out the "Fresh" Score

In [40]:
### Group by "fresh" score

df_rtreview_info['fresh'].value_counts()

totalscores = (33035 + 21397)
totalscores

percentage_of_not_fresh = (33035/totalscores)



percentage_of_fresh = (21397/totalscores)


print("Percentage of Movie Reviewers which are Not Fresh", round((percentage_of_not_fresh * 100)),"%")
print("Percentage of Movie Reviewers which are Fresh",round((percentage_of_fresh * 100)),"%")

Percentage of Movie Reviewers which are Not Fresh 61 %
Percentage of Movie Reviewers which are Fresh 39 %


<h5> Ratings Score

In [41]:
### Group by "rating" score

df_rtreview_info['rating'].value_counts()

3/5       4327
4/5       3672
3/4       3577
2/5       3160
2/4       2712
          ... 
3.0/10       1
3.1          1
7.7          1
7.1          1
4.3/10       1
Name: rating, Length: 186, dtype: int64

<h5> Types of Publishers

In [43]:
### Group by "rating" score

df_rtreview_info['publisher'].value_counts()

eFilmCritic.com          673
EmanuelLevy.Com          591
New York Times           590
Washington Post          565
Entertainment Weekly     541
                        ... 
Fopp.com                   1
Time to Play Magazine      1
BlackNews.com              1
Far Out Magazine           1
Fresh and Frowsy           1
Name: publisher, Length: 1281, dtype: int64

<h2> IMDB DATASET

<h3> Import Movie Basics Folder

In [92]:
conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT *

FROM movie_basics

""",conn)

### IMPORT THE DATA

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


<H3> Different Types of Films

In [27]:
## FOUND OUT HOW MANY TYPES OF GENRE FILMS THERE ARE

conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT Genres ,COUNT(genres)

FROM movie_basics
WHERE runtime_minutes NOT LIKE 'NaN'
GROUP BY genres
ORDER BY COUNT(genres) DESC

LIMIT 20;

""",conn)

### LOTS OF DOCUMENTARY AND DRAMA MOVIES

,genres,COUNT(genres)
0,Documentary,24672
1,Drama,15725
2,Comedy,6413
3,"Comedy,Drama",3163
4,Horror,2975
5,"Biography,Documentary",2052
6,Thriller,1924
7,"Drama,Romance",1806
8,"Documentary,Drama",1515
9,"Comedy,Drama,Romance",1451


<h4> Documentary and Drama seem to be the most popular

<h4> Lets find the average runtime for movies in these genres

<h3> Average Runtime of Drama Movies

In [29]:
### FIND AVERAGE RUNTIME OF MOVIES LIKE "Drama"

conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT AVG(runtime_minutes) AS "Run Time", GENRES AS "Genres", count(GENRES) AS "Amount Of Films"

FROM movie_basics
WHERE GENRES LIKE '%Drama%'
GROUP BY genres
ORDER BY runtime_minutes DESC;

""",conn)

###  

,Run Time,Genres,Amount Of Films
0,107.471154,"Drama,History,War",113
1,101.570122,"Action,Crime,Drama",727
2,98.296296,"Adventure,Drama,Sci-Fi",29
3,134.000000,"Action,Drama,Musical",6
4,110.586207,"Adventure,Drama,History",31
...,...,...,...
222,107.696970,"Action,Drama,Sport",38
223,116.322835,"Action,Drama,Romance",167
224,100.630435,"Action,Drama,Mystery",55
225,103.162791,"Action,Drama,Fantasy",55


<h3> Average Runtime of Documetary Movies

In [70]:
### FIND AVERAGE RUNTIME OF MOVIES LIKE "Documentary"

conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT AVG(runtime_minutes) AS "Run Time", GENRES AS "Genres", count(GENRES) AS "Amount Of Films"

FROM movie_basics
WHERE GENRES LIKE '%Documentary%'
GROUP BY genres
ORDER BY runtime_minutes DESC;

""",conn)

###  

,Run Time,Genres,Amount Of Films
0,74.659574,Documentary,32185
1,86.285714,"Action,Crime,Documentary",8
2,70.500000,"Crime,Documentary,Romance",2
3,72.000000,"Documentary,Sport,Thriller",2
4,120.000000,"Documentary,Musical,Mystery",1
...,...,...,...
201,71.714286,"Crime,Documentary,Drama",124
202,61.000000,"Comedy,Documentary,Sport",10
203,45.000000,"Comedy,Documentary,Sci-Fi",3
204,61.789474,"Biography,Documentary",2115


<h3> Average Runtime of Drama and Documentary Movies

In [47]:
### FIND AVERAGE RUNTIME OF MOVIES LIKE "Drama"

conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT AVG(runtime_minutes) AS "Run Time", GENRES AS "Genres", count(GENRES) AS "Amount Of Films"

FROM movie_basics
WHERE GENRES LIKE '%Drama%' AND GENRES LIKE '%Documentary%'
GROUP BY genres
ORDER BY count(GENRES) DESC;

""",conn)

###  EVEN SOME THAT CONTAIN BOTH!!!  21 Types!! 

,Run Time,Genres,Amount Of Films
0,72.280528,"Documentary,Drama",1554
1,69.875124,"Biography,Documentary,Drama",1028
2,64.732824,"Documentary,Drama,Family",399
3,75.000000,"Documentary,Drama,History",374
4,71.297578,"Comedy,Documentary,Drama",291
5,75.093385,"Adventure,Documentary,Drama",258
6,91.562500,"Documentary,Drama,News",148
7,71.714286,"Crime,Documentary,Drama",124
8,70.813953,"Documentary,Drama,Music",86
9,69.618421,"Documentary,Drama,Sport",76


<H2> Import Movie Ratings

In [90]:
conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT *

FROM movie_ratings

JOIN movie_basics
    USING(movie_id)


""",conn)

### IMPORT THE DATA

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"
...,...,...,...,...,...,...,...,...
73851,tt9805820,8.1,25,Caisa,Caisa,2018,84.0,Documentary
73852,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi"
73853,tt9851050,4.7,14,Sisters,Sisters,2019,NaN,"Action,Drama"
73854,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81.0,Documentary


<h3> What is the Highest Rated Genre?

<h5> Need to determine what genre is rated the best, lets use straight average rating for this model to start

In [91]:
conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT averagerating, numvotes, genres, count(GENRES) AS "Amount Of Films"

FROM movie_ratings

JOIN movie_basics
    USING(movie_id)

GROUP BY genres
ORDER BY averagerating DESC;

""",conn)

### IMPORT THE DATA

,averagerating,numvotes,genres,Amount Of Films
0,9.7,22,"Comedy,Documentary,Sport",4
1,9.4,5,"Documentary,Drama,War",24
2,9.4,5,"Comedy,Documentary,Fantasy",1
3,9.3,50,Musical,132
4,9.3,19,"Documentary,Family,Musical",1
...,...,...,...,...
919,1.8,23,"Adventure,Drama,Musical",2
920,1.7,20,"Family,Sci-Fi",7
921,1.4,28,"Comedy,Musical,Sport",1
922,1.3,33,"Action,Biography,History",5


<h5> Highest Rated Genre Seems to be "Comedy, Documentary, Sports", "Documentary, Drama", "War, Comedy, Documentary, Fantasy", "Musicals" but all have extremely small number of votes and films

<h4> Lets determine what size based on number number of votes is greater than 100

In [123]:
conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT averagerating, numvotes, genres, count(GENRES) AS Amount_Of_Films

FROM movie_ratings

JOIN movie_basics
    USING(movie_id)

WHERE numvotes >= 750

GROUP BY genres
ORDER BY averagerating DESC;

""",conn)

### IMPORT THE DATA

,averagerating,numvotes,genres,Amount_Of_Films
0,9.2,1015,"Comedy,Documentary,History",3
1,8.8,4583,"Drama,Music,Musical",7
2,8.6,1326,Musical,5
3,8.6,1505,"Comedy,Fantasy,Music",3
4,8.5,43409,"Crime,Thriller",58
...,...,...,...,...
510,2.6,4552,"Comedy,Family,Sci-Fi",1
511,2.6,1509,"Action,Sci-Fi",21
512,2.1,3878,"Drama,History,Sport",3
513,1.7,2541,"Comedy,Family,Fantasy",11


<h5> What's the runtime length of these movies

In [106]:
conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT averagerating as "Average Ratings", numvotes as"Number of Votes", genres as "Genres", runtime_minutes as "Run Time"

FROM movie_akas

JOIN movie_basics
    USING(movie_id)

JOIN movie_ratings
    USING(movie_id)

GROUP BY genres
ORDER BY runtime_minutes DESC

""",conn)

### IMPORT THE DATA

,Average Ratings,Number of Votes,Genres,Run Time
0,6.8,13,"Adventure,Drama,Fantasy",270.0
1,4.5,245,"Action,Western",180.0
2,7.4,8,"History,Musical,Romance",174.0
3,7.9,539,"Fantasy,Musical,Sci-Fi",172.0
4,7.1,113,"Biography,Crime,Thriller",171.0
...,...,...,...,...
910,4.6,273,"Action,Comedy,Mystery",NaN
911,7.4,50,"Action,Comedy,Drama",NaN
912,5.8,13,"Action,Comedy",NaN
913,3.4,28,"Action,Adult,Comedy",NaN


In [124]:
conn = sqlite3.connect("zippedData/im.db")

pd.read_sql("""
SELECT *
FROM movie_akas

JOIN movie_basics
    USING(movie_id)
    
JOIN movie_ratings
    USING(movie_id)

WHERE region = "US"


ORDER BY numvotes DESC

LIMIT 20;

""",conn)

### Number of Votes in the US is greater in "Action", "Adventure", "Sci-Fi"

,movie_id,ordering,title,region,language,types,attributes,is_original_title,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt1375666,24,Inception,US,None,imdbDisplay,None,0.0,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
1,tt1375666,30,Oliver's Arrow,US,None,None,fake working title,0.0,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
2,tt1375666,42,Inception: The IMAX Experience,US,None,None,IMAX version,0.0,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
3,tt1375666,8,Inception,US,en,None,None,0.0,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
4,tt1345836,14,Arkham,US,None,working,None,0.0,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
5,tt1345836,16,Gotham,US,None,working,None,0.0,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
6,tt1345836,17,Batman 3,US,None,working,None,0.0,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
7,tt1345836,28,The Dark Knight Rises: The IMAX Experience,US,None,None,IMAX version,0.0,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
8,tt1345836,48,T.D.K.R.,US,None,None,informal short title,0.0,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
9,tt1345836,9,Magnus Rex,US,None,None,fake working title,0.0,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769


In [ ]:
### DO SOME JOIN STAT

In [ ]:
### find the highest rated movie based on genre, and runtime of the movie, - number of votes, genre, year released, etc